In [196]:
import pandas as pd
import os
import matplotlib.pyplot as pyplt
import seaborn as sns
sns.set()
import math

def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [197]:
coord_dir = 'C../blackpanther/index'
output_file = 'coord_user_weighted.csv'

bothunter_folder = '../bothunter output/reopen'
bothunter_file = 'botoutput.csv'
bot_file = os.path.join(bothunter_folder, bothunter_file)

In [169]:
#df_bots = df_bots[df_bots['probability'] != 'probability']

In [170]:
#df_bots['IS_BOT'] = df_bots['probability'].apply(lambda x: True if float(x) > 0.7 else False)

In [198]:
df_pairs = pd.read_csv(os.path.join(coord_dir, 'coord_pairs_output_simple_multiply.csv'))
df_bots = pd.read_csv(bot_file)
df_bots['IS_BOT'] = df_bots['probability'].apply(lambda x: True if float(x) > 0.7 else False)

In [199]:
df_pairs_unique_users = df_pairs['Source Node ID'].unique().tolist()

In [200]:
def get_pair_coord_total(df):
    link_value = 0
    counter = 0
    user_coord_value = 0
    
    for idx, row in df.iterrows():
        link_value += row['coord_index']
        counter += 1
            
        user_coord_value += row['Link Value'] * row['coord_index']
    
    return user_coord_value, counter

In [201]:
df_user_coord_arr = []

for user in df_pairs_unique_users:
    df_subset = df_pairs[(df_pairs['Source Node ID'] == user)]
    coord_total, counter = get_pair_coord_total(df_subset)
    
    coord_sigmoid = sigmoid(coord_total)
        
    obj = {'user': user, 'coord_index': coord_total, 'coord_sigmoid': coord_sigmoid, 'counter': counter}
    df_user_coord_arr.append(obj)

In [202]:
df_user_coord = pd.DataFrame(df_user_coord_arr)
df_user_coord.head()

,user,coord_index,coord_sigmoid,counter
0,DeplorabCthulhu,4.000000,0.982014,1
1,archangelscottt,5.025907,0.993477,1
2,ResistCovid19,5.025907,0.993477,1
3,shercares,5.025907,0.993477,1
4,Kris_Sacrebleu,7.108808,0.999183,1


In [203]:
df_merge = df_user_coord.merge(df_bots, how='outer', left_on='user', right_on='screen_name')

In [119]:
df_merge.head()

,user,coord_index,coord_sigmoid,counter,status_id,user_id,screen_name,prediction,probability,IS_BOT
0,TheUmerAwan,7.457944,0.999423,1.0,1322038622011404289,2921379161,TheUmerAwan,False,0.048418,False
1,KashafN42768705,4.149533,0.984473,1.0,1322171400753418240,1060449656629551109,KashafN42768705,False,0.392940,False
2,KashafN42768705,4.149533,0.984473,1.0,1322170197894156288,1060449656629551109,KashafN42768705,False,0.392940,False
3,aqdushira,6.336449,0.998233,1.0,1322171949372579841,1673070157,aqdushira,False,0.100962,False
4,aqdushira,6.336449,0.998233,1.0,1322170209755693056,1673070157,aqdushira,False,0.100962,False


In [176]:
df_merge.to_csv(os.path.join(coord_dir, output_file), index=False)

In [159]:
df_user_coord['coord_index'].describe()

count    3563.000000
mean        2.571227
std         8.751025
min         0.000000
25%         1.004608
50%         1.004608
75%         1.004608
max       434.000000
Name: coord_index, dtype: float64

In [160]:
df_user_coord['coord_index'].median()

1.0046082949308757

In [204]:
df_merge_bots = df_merge[df_merge['IS_BOT'] == True]
df_merge_humans = df_merge[df_merge['IS_BOT'] == False]

In [205]:
df_merge_bots['coord_index'].describe()

count    3572.000000
mean        7.375041
std         5.395809
min         4.000000
25%         5.025907
50%         5.025907
75%        11.398964
max       109.699482
Name: coord_index, dtype: float64

In [206]:
df_merge_humans['coord_index'].describe()

count    6677.000000
mean        7.218327
std         5.418153
min         4.000000
25%         4.000000
50%         5.025907
75%        11.398964
max       116.974093
Name: coord_index, dtype: float64